# **Convolutional Neural Network - CNN**

## Objectives

* To classify the image set
* To test the performance of the classification using resizing vs resizing + padding

## Inputs

* CSVs outputted from ETL for resizing and resizing + padding

## Outputs

* Write here which files, code or artefacts you generate by the end of the notebook 

## Additional Comments

* If you have any additional comments that don't fit in the previous bullets, please state them here. 



---

## Import Libraries

In [1]:
import numpy as np
import pandas as pd
import os
import base64
import cv2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

---

# Change working directory

* We are assuming you will store the notebooks in a subfolder, therefore when running the notebook in the editor, you will need to change the working directory

We need to change the working directory from its current folder to its parent folder
* We access the current directory with os.getcwd()

In [2]:
current_dir = os.getcwd()
current_dir

'c:\\Users\\Eddie\\Documents\\CodeInstitute Workspace\\Capstone Project\\jupyter_notebooks'

We want to make the parent of the current directory the new current directory
* os.path.dirname() gets the parent directory
* os.chir() defines the new current directory

In [3]:
if os.path.basename(current_dir).lower() == "jupyter_notebooks":
    os.chdir(os.path.dirname(current_dir))
    print("You set a new current directory")

You set a new current directory


Confirm the new current directory

In [4]:
current_dir = os.getcwd()
current_dir

'c:\\Users\\Eddie\\Documents\\CodeInstitute Workspace\\Capstone Project'

---

# Load CSV files

In [5]:
df_resize = pd.read_csv("./Data/Processed/df_resized.csv")
df_resize_padding = pd.read_csv("./Data/Processed/df_resized_padded.csv")

# Display the first few rows of each DataFrame
df_resize.head()
df_resize_padding.head()

,subfolder,image,grey_image,luminance,contrast
0,anger,iVBORw0KGgoAAAANSUhEUgAAAlgAAAJYCAIAAAAxBA+LAA...,iVBORw0KGgoAAAANSUhEUgAAAlgAAAJYCAAAAACbDccAAA...,202.218752,42.869499
1,anger,iVBORw0KGgoAAAANSUhEUgAAAlgAAAJYCAIAAAAxBA+LAA...,iVBORw0KGgoAAAANSUhEUgAAAlgAAAJYCAAAAACbDccAAA...,134.196622,101.303135
2,anger,iVBORw0KGgoAAAANSUhEUgAAAlgAAAJYCAIAAAAxBA+LAA...,iVBORw0KGgoAAAANSUhEUgAAAlgAAAJYCAAAAACbDccAAA...,109.144209,51.433215
3,anger,iVBORw0KGgoAAAANSUhEUgAAAlgAAAJYCAIAAAAxBA+LAA...,iVBORw0KGgoAAAANSUhEUgAAAlgAAAJYCAAAAACbDccAAA...,140.261702,89.338355
4,anger,iVBORw0KGgoAAAANSUhEUgAAAlgAAAJYCAIAAAAxBA+LAA...,iVBORw0KGgoAAAANSUhEUgAAAlgAAAJYCAAAAACbDccAAA...,143.852701,81.434234


In [6]:
#convert back to numpy arrays
def base64_to_cv2_array(b64_string):
    # Decode base64 → bytes
    img_bytes = base64.b64decode(b64_string)

    # Convert bytes → 1D uint8 array (image buffer)
    img_array = np.frombuffer(img_bytes, dtype=np.uint8)

    return cv2.imdecode(img_array, cv2.IMREAD_UNCHANGED)

df_resize['image'] = df_resize['image'].apply(lambda x: base64_to_cv2_array(x))
df_resize['grey_image'] = df_resize['grey_image'].apply(lambda x: base64_to_cv2_array(x))

df_resize_padding['image'] = df_resize_padding['image'].apply(lambda x: base64_to_cv2_array(x))
df_resize_padding['grey_image'] = df_resize_padding['grey_image'].apply(lambda x: base64_to_cv2_array(x))

---

# CNN

Encode labels and split data into training, validation, and testing

In [7]:
#encode labels
encoder = LabelEncoder()
df_resize['encoded_labels'] = encoder.fit_transform(df_resize['subfolder'])
df_resize_padding['encoded_labels'] = encoder.fit_transform(df_resize_padding['subfolder'])

#split data into training, validation, and testing
train_df_resize, temp_df_resize = train_test_split(
    df_resize, test_size=0.4, random_state=1, stratify=df_resize['encoded_labels']
)
val_df_resize, test_df_resize = train_test_split(
    temp_df_resize, test_size=0.75, random_state=1, stratify=temp_df_resize['encoded_labels']
)

In [ ]:
df_resize['encoded_labels'].value_counts()

Colour images - resized

In [8]:


model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(600, 600, 3)),
    MaxPooling2D((2,2)),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D((2,2)),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(6, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

stacked_images = np.stack(train_df_resize['image'].values).astype('float16')/255.0
stacked_labels = np.stack(train_df_resize['encoded_labels'].values)
stacked_val_images = np.stack(val_df_resize['image'].values).astype('float16')/255.0
stacked_val_labels = np.stack(val_df_resize['encoded_labels'].values)
stacked_test_images = np.stack(test_df_resize['image'].values).astype('float16')/255.0
stacked_test_labels = np.stack(test_df_resize['encoded_labels'].values)

#model.fit(stacked_images, stacked_labels, epochs=5, validation_data=(stacked_val_images, stacked_val_labels))

#test_loss, test_accuracy = model.evaluate(stacked_test_images, stacked_test_labels)

c:\Users\Eddie\Documents\CodeInstitute Workspace\Capstone Project\venv\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [9]:
model.fit(stacked_images, stacked_labels, epochs=4, validation_data=(stacked_val_images, stacked_val_labels))

test_loss, test_accuracy = model.evaluate(stacked_test_images, stacked_test_labels)

Epoch 1/4
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 37s/step - accuracy: 0.1669 - loss: 34.1920 

MemoryError: Unable to allocate 494. MiB for an array with shape (120, 600, 600, 3) and data type float32

---

NOTE

* You may add as many sections as you want, as long as it supports your project workflow.
* All notebook's cells should be run top-down (you can't create a dynamic wherein a given point you need to go back to a previous cell to execute some task, like go back to a previous cell and refresh a variable content)

---

# Push files to Repo

* In cases where you don't need to push files to Repo, you may replace this section with "Conclusions and Next Steps" and state your conclusions and next steps.